In [3]:

import sqlite3
import pandas as pd

In [4]:

con = sqlite3.connect(':memory:')

In [5]:

con = sqlite3.connect(':memory:')
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [6]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

Categories
Customers
Employees
OrderDetails
Orders
Products
Shippers
Suppliers

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [5]:
pd.read_sql(
    '''
    SELECT CustomerName,
        Country,
        Address
    FROM Customers
    WHERE Country IN ('Germany','France') OR City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [6]:
pd.read_sql(
    '''
    SELECT 
        count(Country) AS cnt,
        Country
        
    FROM Customers
    group by Country
    order by cnt desc
    LIMIT 3
    ''',
    con,
)

,cnt,Country
0,13,USA
1,11,Germany
2,11,France


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [65]:
pd.read_sql(
    '''
    SELECT
      s.ShipperName,
      o.OrderDate
    FROM Orders o --Details
    join Shippers s on o.ShipperID = s.ShipperID
    WHERE OrderID = (
    SELECT OrderID
    FROM (
    SELECT OrderID,
        row_number() over() as nom
    FROM Orders
    order by OrderDate desc)
    WHERE nom = 10
    )
   ''',
    con,
)


,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [8]:
data = pd.read_csv('OrderDetails.csv', sep = '\t')
data.to_sql('OrderDetails', con, index=False, if_exists='replace')
pd.read_sql(
    '''
    SELECT
      ProductName,
      Price
    FROM OrderDetails o
    join Products p on o.ProductID = p.ProductID
    WHERE OrderID = (
    SELECT
      OrderID
    FROM (
    SELECT
      OrderID,
      sum(o.Quantity * p.Price) AS TotCena
    FROM OrderDetails o
    join Products p on o.ProductID = p.ProductID
    group by o.OrderID 
    order by TotCena desc
    LIMIT 1
    )
    )
   ''',
    con,
)

,ProductName,Price
0,Sir Rodney's Marmalade,81.0
1,Côte de Blaye,263.5
2,Camembert Pierrot,34.0
3,Mozzarella di Giovanni,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [52]:
pd.read_sql(
    '''
    SELECT
      OrderID,
      ProductName,
      Quantity
    FROM (SELECT
        *
    FROM OrderDetails o
    WHERE ProductID = (
    SELECT
      ProductID
    FROM (
    SELECT
      ProductID, sum(Quantity) as kol
    FROM OrderDetails
    group by ProductID
    order by kol desc
    limit 1
    ))) o
    join Products p on o.ProductID = p.ProductID
   ''',
    con,
)

,OrderID,ProductName,Quantity
0,10253,Gorgonzola Telino,20
1,10272,Gorgonzola Telino,40
2,10273,Gorgonzola Telino,15
3,10325,Gorgonzola Telino,4
4,10335,Gorgonzola Telino,25
5,10342,Gorgonzola Telino,56
6,10356,Gorgonzola Telino,30
7,10359,Gorgonzola Telino,70
8,10363,Gorgonzola Telino,20
9,10374,Gorgonzola Telino,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [104]:
pd.read_sql(
    '''
    SELECT
      SupplierName,
      Country,
      ContactName,
      Phone
    FROM (SELECT
      *, count(OrderID) AS Kol
    FROM OrderDetails o
    join Products p on o.ProductID = p.ProductID
    group by SupplierID
    order by Kol desc
    limit 5) o
    join Suppliers s on o.SupplierID = s.SupplierID
   ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [42]:
pd.read_sql(
    '''
    SELECT Country, CategoryName, SUM
    FROM (
    SELECT Country, CategoryID, od.Quantity * p.Price AS SUM  --
    FROM (
    SELECT *
    FROM (
    SELECT * --, count(OrderID) AS Kol
    FROM OrderDetails od
    join Orders o on od.OrderID = o.OrderID) od
    join Customers c on od.CustomerID = c.CustomerID
    WHERE Country = 'Brazil') od
    join Products p on od.ProductID = p.ProductID
    
    ) od
    join Categories c on od.CategoryID = c.CategoryID
    order by SUM desc
   ''',
    con,
)

,Country,CategoryName,SUM
0,Brazil,Beverages,10540.00
1,Brazil,Dairy Products,2380.00
2,Brazil,Produce,2120.00
3,Brazil,Meat/Poultry,1940.00
4,Brazil,Produce,1915.20
5,Brazil,Meat/Poultry,1856.85
6,Brazil,Produce,1855.00
7,Brazil,Confections,1620.00
8,Brazil,Dairy Products,1461.60
9,Brazil,Dairy Products,1100.00


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [53]:
pd.read_sql(
    '''
    SELECT  max(Sum) - min(Sum) AS raz
    FROM (
    SELECT *, sum(od.Quantity*p.Price) AS Sum
    FROM OrderDetails od
    join Products p on od.ProductID = p.ProductID
    join Orders o on od.OrderID = o.OrderID
    group by od.OrderID) od
    join Customers c on od.CustomerID = c.CustomerID
    WHERE Country = 'USA'
   ''',
    con,
)

,raz
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [39]:
pd.read_sql(
    '''
    SELECT  count() as cnt , FirstName || ' ' || LastName as FIO
    FROM Orders o
    join (SELECT
    *, substr(BirthDate,1,2) as day, substr(BirthDate,4,2) as month, substr(BirthDate,7,4) as yahr
    FROM Employees
    order by yahr desc, month desc, day desc
    limit 3) e on o.EmployeeID=e.EmployeeID
    group by FIO
    order by cnt desc
    
   ''',
    con #, parse_dates={"BirthDate": {"format": "%d.%m.%Y"}}
)

,cnt,FIO
0,31,Janet Leverling
1,29,Nancy Davolio
2,6,Anne Dodsworth


10. Сколько банок крабового мяса всего было заказано.

In [13]:
pd.read_sql(
    '''
    SELECT sum(Quantity*24) AS cnt_crab
    FROM OrderDetails
    WHERE ProductID = (SELECT ProductID
    FROM Products p
    WHERE ProductName like '%crab meat%')
 ''',
    con
)

,cnt_crab
0,6144
